# The Midwest underwater

A look at 2019 floods in South Dakota, USA

Elsa Culler  
Nate Quarderer  
2025-05-20

## Set up

To get started on this notebook, you’ll need to restore any variables
from previous notebooks to your workspace.

In [1]:
%store -r

# Import libraries
import pandas as pd #work with tabular data
import geopandas as gpd #work with vecor spatial data
import hvplot.pandas #to make interactive plots

## STEP 4: Analyse the flood

As scientists and engineers, we are interested in not just describing a
flood, but in understanding how often we would expect an event that
severe or extreme to happen. Some applications we need this information
for include:

-   Designing and developing engineering standards for bridges and roads
    to withstand flooding
-   Choosing the capacity of water treatment plants to accommodate flood
    waters
-   Computing flood risk maps and choosing where to build
-   Determining flood insurance rates

The exceedance probability is a simple, data-driven way to quantify how
unusual a flood is and how often we can expect similar events to happen.
We calculate exceedance probability by counting how many years with
floods the same size or larger have been recorded, or ranking the and
dividing by the number of years we have records for:

$$P_e = \frac{\text{Annual peak flow rank}}{\text{Years of record}}$$

This value tells us historically what the likelihood was of a flood of a
certain size or larger each year, or the **exceedance probability**. We
can also express how unusual a flood is with the **return period**, or
an amount of time during which we’d expect there to be about one flood
the same size or larger. The return period is the reciprocal of the
exceedance probability:

$$R = \frac{\text{Years of record}}{\text{Annual peak flow rank}}$$

As an example – suppose a streamflow of $10000$ cfs occurs $4$ times
over a 100-year record. The exceedance probability would be
$\frac{4}{100} = .25$ and the return period would be 25 years.

There are advantages and disadvantages to this method of calculating the
**exceedance probability**. On one hand, we are not making any
assumptions about how often floods occur, and there is no way to
extrapolate to a size of flood that has never been observed. On the
other hand, we can’t incorporate any information about how often floods
occur nearby or in other locations, and the data record for streamflow
is often less than the desired lifetime of the built environment.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-read"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Read More</div></div><div class="callout-body-container callout-body"><p>You can learn more about exceedance probabilities and return periods
in <a
href="https://www.earthdatascience.org/courses/use-data-open-source-python/use-time-series-data-in-python/floods-return-period-and-probability/">this
textbook page on the subject</a></p></div></div>

Let’s start by accessing and plotting ALL the data available for this
site. Then we’ll use a return period **statistic** to quantify how
unusual it was.

### Visualize all the streamflow data

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>In the cell below, plot the entire time series of streamflow data,
without any parameters.</p></div></div>

In [3]:
# Plot the entire streamflow time series

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Do you notice anything about this plot?</p></div></div>

First things first – this plot looks a little fuzzy because it is trying
to fit too many data points in a small area. There aren’t enough pixels
in this plot to represent all the data points! One way to improve this
is by **resampling** the data to **annual maxima**. That way we still
get the same peak streamflows, but the computer will be able to plot all
the values without overlapping.

> **Tip**
>
> **Resampling** means changing the time interval between time series
> observations - in this case from daily to annual.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-read"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Read More</div></div><div class="callout-body-container callout-body"><p>Read about <a
href="https://www.earthdatascience.org/courses/use-data-open-source-python/use-time-series-data-in-python/date-time-types-in-pandas-python/resample-time-series-data-pandas-python/">different
ways to resample time series data in your textbook</a></p>
<p>You can use a <a
href="https://pandas.pydata.org/docs/dev/user_guide/timeseries.html#timeseries-offset-aliases">list
of <strong>offset aliases</strong></a> to look up how to specify the
final dates. This list is pretty hard to find - you might want to
bookmark it or check back with this page if you need it again.</p></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Resample your <code>DataFrame</code> to get an annual maximum:</p>
<ol type="1">
<li>Replace <code>dataframe_name</code> with the name of your
<code>DataFrame</code>.</li>
<li>Replace <code>offset_alias</code> with the correct offset alias from
the <a
href="https://pandas.pydata.org/docs/dev/user_guide/timeseries.html#timeseries-offset-aliases">pandas
documentation</a></li>
<li>Save the results to a new, descriptive variable name, and display
the results of the resampling.</li>
</ol></div></div>

In [5]:
# Resample to annual maxima
dataframe_name.resample(offset_alias).max()

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Plot your resampled data.</p></div></div>

In [7]:
# Plot annual maximum streamflow values

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Write a headline and 2-3 sentence description of your plot. What is
your visual estimate of the return period was for the flood in 2019?</p></div></div>

YOUR HEADLINE AND PLOT DESCRIPTION HERE

### Select relevant data

When calculating exceedance probabilities, we are making an assumption
of **stationarity**, meaning that all the peak streamflows are drawn
from the same **probability distribution**. Put another way, we only
want to include data from years where the conditions on the river are
similar to what they are now.

Did you notice that the streamflow values from before 1950 or so? You
should investigate any obvious causes of that discrepancy so we know if
the pre-1950 data is relevant to current conditions.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>What are some possible causes for peak streamflows to decrease
systematically?</p></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-info"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Climate Change and Stationarity</div></div><div class="callout-body-container callout-body"><p>One of the problems with adapting to climate change is that we can no
longer assume stationarity in a lot of contexts. As scientists, we don’t
yet have standard methods for incorporating climate change into flood
return period calculations. You can read more about the debate of
stationarity, climate change, and return periods in <a
href="https://www.science.org/doi/10.1126/science.1151915">a paper
called ‘Stationarity is Dead’</a> and the many related response
papers.</p></div></div>

It turns out that construction on the Oahe dam on the Cheyenne River was
started in 1948. We therefor don’t want to include any streamflow
measurements before that date, because the Cheyenne River now as a much
different flood response due to the dam. Dams tend to reduce peak
streamflow, depending on how they are managed, but can cause other
problems in the process.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-read"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Read More</div></div><div class="callout-body-container callout-body"><p>Learn more about the Oahe Dam on <a
href="https://en.wikipedia.org/wiki/Oahe_Dam">its Wikipedia page</a>.
You can also find some local perspectives on the dam in some of the
articles about the 2019 flood at the beginning of this coding
challenge.</p></div></div>

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Remove years of data before the construction of the Oahe Dam. You can
use a colon inside the square brackets of the <code>.loc</code>
attribute to show that you would like all dates after a certain value,
e.g. <code>'1950':</code></p></div></div>

In [9]:
# Select data from after dam construction

### Calculate the exceedance probability and return period for 2019

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-extra"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Looking for an Extra Challenge?</div></div><div class="callout-body-container callout-body"><p>Calculate the <strong>exceedance probability</strong> and
<strong>return period</strong> for each year of the
<strong>annual</strong> data, and add them as columns to your
DataFrame.</p>
<ol type="1">
<li>Replace <code>df</code> with the name of your <strong>annual
maximum</strong> <code>DataFrame</code>.</li>
<li>Replace <code>col</code> with the name of your streamflow
column</li>
<li>Calculate the return period using Python mathematical operators</li>
</ol></div></div>

> **Tip**
>
> When you use a Python mathematical operator on a `pandas.DataFrame`
> column, Python will do the calculation for every row in the
> `DataFrame` automatically!

> **Tip**
>
> When you rank the floods in your `DataFrame` with the `.rank()`
> method, you will need the
> ascending=False`parameter, by default the largest floods will have the higher number. We use`ascending=Falsa\`
> to reverse the rankings, since higher rank should be lower exceedence
> probability.

In [11]:
df['exceed_prob'] = (df.rank(ascending=False).col / len(df))
df['return_period'] = 

peaks_df

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Select only the value for 2019.</p>
<ol type="1">
<li>Replace <code>dataframe_name</code> with the name of your
<code>DataFrame</code></li>
<li>Inside the square brackets, type the year you want to select (2019).
Make sure to surround the year with quotes, or Python will interpret
this as a <strong>row number</strong>.</li>
</ol></div></div>

In [13]:
dataframe_name.loc[]

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>What is the exceedance probability and return period for the 2019
floods on the Cheyenne River?</p></div></div>

PROBABILITY AND RETURN PERIOD HERE

## Wrap up

Don’t forget to store your variables so you can use them in other
notebooks! This code will store all your variables. You might want to
specify specific variables, especially if you have large objects in
memory that you won’t need in the future.

In [15]:
%store

Finally, be sure to `Restart` and `Run all` to make sure your notebook
works all the way through!